In [7]:
# !pip install mpldatacursor # (https://github.com/joferkington/mpldatacursor)
# !pip install calmap # (https://github.com/martijnvermaat/calmap/blob/master/calmap/__init__.py)

In [8]:
import sys; sys.path.append("../../..")
from notebook_imports import *
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.set_option("display.max_rows", 999)
pd.get_option("display.max_rows")

999

In [9]:
%%javascript

// Javascript learnings ...

// Production librarys for JS can be found here: https://cdnjs.cloudflare.com
    // Where else can they be found?

// H not being defined was because a box couldnt be drawn since the div it wanted to draw on had no space ...
    // What are the different dom events?
    // What are some of the assumptions ui engineers make? In this case, the assumption was that the div 
    // - [ ] Open an issue with cytoscape showing that it crashes/errors if the dif you are trying to dislpay to has no space.

// Figure out what js gets imported by default ...
// What is the best way to capture size from a reactive perspective - no pixels? Or atleast at the wee end and then have everything be relative to an anchor on the screen?

// Degree on a node is the number of connections it has?
//     https://www.biostars.org/p/280679/

<IPython.core.display.Javascript object>

In [10]:
import ipywidgets as widgets
from traitlets import Unicode, validate, Dict, List
import json

class CytoscapeWidget(widgets.DOMWidget):
    _view_name = Unicode('CytoscapeView').tag(sync=True) # This is the name of the Javascript View
    _view_module = Unicode('cytoscapewidget').tag(sync=True) # This is the name of the defined java script method
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = Unicode('Hello World!').tag(sync=True)
    domid = Unicode('cy').tag(sync=True)
    elements = Unicode('{}').tag(sync=True)
    sytleFile = Unicode('https://js.cytoscape.org/demos/labels/cy-style.json').tag(sync=True)

In [11]:
%%javascript

require.undef('cytoscapewidget');

require.config({
  paths: {
    'cytoscape': 'https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.2.16/cytoscape.min'
  },
  shim: {
    'cytoscape': {
        exports: 'cytoscape'
    }
  },
  waitSeconds: 30
});

define('cytoscapewidget', ["@jupyter-widgets/base", "cytoscape"], function(widgets, cytoscape) {
    var CytoscapeView = widgets.DOMWidgetView.extend({
        _render: function() {
            // Python sends over to the JavaScript Json STRINGS! They dont actually get decoded into the json types ...
            var elements = JSON.parse(this.model.get('elements'));
            var elementId = this.model.get('domid')
            // We have to use .get on model because its an attribute (model.attriubte.styleFile). It looks like .get knows how to look in the attributes ...
            fetch(this.model.get('sytleFile'))
            .then(function(res) {
                return res.json()
            })
            .then(function(style) {
                var fullStyle = style.slice(2,).concat([
                    {
                        selector: 'node',
                        style: {
                            'height': '128',
                            'width': '128',
                            'label': 'data(label)'
                        }
                    },
                    {   
                        selector: 'edge',
                        style: {
                            'label': 'data(label)',
                            'curve-style': 'haystack',
                            'haystack-radius': 0,
                            'width': '32',
                            'opacity': 0.5,
                            'line-color': '#a8eae5'
                        }
                    },
                    {
                        selector: '.profile',
                        style: {
                            'background-image': 'https://cdn3.iconfinder.com/data/icons/vector-icons-6/96/256-128.png',
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.assets',
                        style: {
                            'background-image': 'https://png.icons8.com/dusk/2x/get-cash.png',
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.preferences',
                        style: {
                            'background-image': "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAMAAAD04JH5AAAAeFBMVEUjHyD///8lISL+/v709PT19fX7+/v4+Pg+Ojs7ODji4uLV1dUmIiMtKirw8PBGQkOPjY1ZVle8urtwbW6zsrJjYGGXlZVKR0edm5tSTk8zLzDEw8Srqqrc3NyGhIXm5uZraWnPzs59ent/fn3Av79mZGSbmZqmpaUKLSEhAAAHyklEQVR4nL2bi5KqMAyGy1VAAQHvCiy6ru//hqeAXNombeGgndmZndjmiwjp39ASY1bziND8eZ4Ib7DqprJZGzGAjas3lrPxAej5iEQ+IeUcPheA5dbNUtrWZt24ACq9sZyNTOzf2lKAb6Zz+EwAlrtarYT+kO0F8M2T3ljONgrAWtWN6w/btgDfLPTGcjYysX9ruwN8M5vDHwKwVo7jCP0R2x7gm4neWM5Gxp85XH8Hsx0BPon1xnIPJRn4ttDfthHbBeCTUG8sy7fIm78SYrNxHyuIT4iNsxwkJqsNoH42eH7g3SPsOwQgn+QY3y28AOS7bQDNs7ni+fSuKpFrWIF8ckX4Eb1nvTXEfwfguiDfNL01yDd2IJ/cYH5eT53UF8BvA7DqACA+va9SiG+cQD45gPw0bPv5ayEn1CmZ0BtByNcdnw7cdp/0foPXMwT5JHy+Ap5vbc2unx9wfBqA8Q4A49P2tMf88m+cAoA58XjIx3z7Oeo3jqDJiTW3CUDCJyQOOr6d7hkhAs3JprlJdr2/4Mj0GyKgOek9J9UBSPmEXK4NP3qB+Q+yxTe3cXW9cP26COqc+L7vaACGnE/F1h/l33gZiPNp8+qrcNoI/doI6Pfp5wRekgF86iP7PaMsxHa+ZlC/OoL6evZzEtHhL2nzuZxIvsynEbA5kXybT2hGGuepcQD2V/g0I43nmLEmdO5f4Zt3RBM6dpR8g5+4BhRAc2+Wnp6P/7F5kQEFQDVZfW+uQx0f/2MLmSle1ISpqfbxX7Ydyxc14faz/ILFA5rQen6Sv2dX+aAmtOPP8WOb5cOaMLh8ih/mHB/RhL+bz/DNX4Yv0YR/en79JHsUxT1LfL2YDhwf1YSWkyn5cZFGw7wapUWs5Gc8H9OE1O+vnL+5V4agv6t7KOX365aWgWvC2q9U/4RFaYj8+jpsQ9lvd7bGfFQT1n5vMv5P2fcT1jr5j+y3240YqCZs9K9Ef3ZrJWz9vQvxsZ479MM0YeP3JfGxlvNtZ+1hY7vFo0wTtusPXP8nkYpvGVGC8UlsiWOJ6CNF+Zmr5tPEmiF8QlJxLBF97DF+osWnESQInyTiWCL4KLH1n6dx/dv2vovFZ9IshbFE8PGH8EPl/TeqSWA56SCMJYIPbP2teP5YW4rkxFgYy2lCgylAMflnCt8wfpCcuFbWCV8wP5TkPzAnhnBOPCnrhE+QT4ppfMcuEE3G9mM14Sqodie4/hNGE/l2AOfk8LSrglXXr1FEVrROX9v7/gjWX9/tPpVvOw+I/26X4/6+faVV6VrE09Jf1WS+Vcn8dbaN330o58fT+YYR4/56W/+fnF/fgpP5RqHm9wEo+DQJTecbv7p8ota/0Qy+Yevy+SZ+5s/hG4a/FJ8ks/hGshSfZLP4xn0pvvmYxW8eg0X4ZjGLX5caluGbxSw+DWAhPltX089JxVJ8M1OyQFu2FJ8ks/hOshSf2R4yISf5S/FpKp7DLzeL8Uk6g2+ny/FJMYNvFxoMTX73Zn7anBRrMOifv9HJV9V0/lrnO1JJZrhllZ6KLIllder7ZL5VENxfGCdZQfVxLUpHdUI7v94OIRgvleUT+RH8fcLD7Zo39VKsTviE+IRsp2qSLcgn/cIErRO+4N8rDKbxyxDkk9OID9cJA+R++ZmmCTKY3+2ykNUJj8gzsZvCTxF+/zxL6oQH5Jnk3rVI+XkI84c9DpI6Yb5BcgLztknKHxcaGb7Z11hldUL03d3ofZuU7yYIv53XlXXCHcIfynQKfobxm0lNXSe0YoTfFyrl1x/9/k2hUqNOaNwwPiFeruLn2O9P2wsYK9YJ6TWU+AgV69QUu/9p81fAWLFOaIw3KgE5IZPkxAjLP027QbGLdcL6/zPOJ2ZYlMj8s0Xyb9vOFnTthDph064Sfp2THpXIrx6K985XiI/sJwT3nzC2uPi1B345vLTCr10G8eH9hCclv/nfT+7Fdku1jK+lf82DeO/A+wmv4v4fmV9t2ybl+fB+woDf/7QQ3zQvOccH3x3bx6l+9W3My2vk3bGFaLJlbM/hN8A04faTfKotR3xQEzbrqc/x+7eXmCZE37UsZmuFFaYJo/jT/PbdF6YJ3f3n+bWwQjXhlLraf9gemCZ0vK/wiV9imjDwv8Jf45pw7X+bz2tCGsF3+YImXPuQj+x6FmwK1vma6fBFTRiIe3JMuqCydh5jU/D9G9VfB3GNJfABTTiOoPEbtvuP3Jv2Hq/41erf9ML1E/mQJhwiaPzGw7mFlK7alHwzSfv1R84KNYAPasJg/K5lz+w/Kw+xnB8fmD0+9lPBhzVh0L9ONAtmQO13fdqHMD/cn/KhX8uytjI+esakjqD5+dn9j51fZI/Dge/XtG5jD8THz5i0OdFDahI7kN8d8eDXP+3GHpCPnzFpcmLC7n8d/F7h3/8K8tv1MsxHz5g0OfHu8rbORw7f/znMp9/wAfEl+wkbH9gZH9ockN8edILXziXEl+wnVNUELxA/1Bs78CX7CVU+jgB/4jt+2X5C9Tm5PcCvK1C6Z+ykdUKdc4IZwCfZtDOLWJ1Q65wk+E60mHJmc4XVCfV8vKA56TSHL5w71jt3m0Jz4k73zC5jm3fuuILm5GqBc8e6Z6dLaE4u5/DZALTPjrvg4Xe9sZyNTOz/tvliAP4svvEPrvyFJpxHCqYAAAAASUVORK5CYII=",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.usecases',
                        style: {
                            'background-image': "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAMAAAD04JH5AAAAk1BMVEX////09PT+/v719fVhYmIlpppfYGD7+/v4+PgnoJXr6+vu7u7k5OTc3d3Q0NCNjo5FtKmw39uEzcaY1c/T1NQsqpzn9fS+5OFbvLPL4+Bcta7FxsWw2NTT7eqTzMa22tfd6uml2tWCxcBoaWnL6ufv+PdpwbiAgICbm5txdHS8vb2srKw9qZ3f8vDk7u2BgYF1xr9QpK+eAAAEFklEQVR4nO2b63KqMBRGcxERBI/3G/ZqPdZW2/P+T3eyAyqRKKGTTZwO+0+d3QzfGghhtRJC7q0YlIPeneQzH4rV3ruf/E6nUxiP3zv9rgPFau/dTz7nvDAev6f8jl+M5/i9fD4tjKcUu0fYMb+j4cXPPy5IcG84yfdTAHlvXs6XWvIzAN93lZ8CMABwkC9WRLkkMlZcr+vJFwAkA3CTnz0TBICDfLEiHp9Jl3JQUz4/zbvLpamWfHr9mVBPPnWdX3gmNPlNfpMPTlY23o97ohKonlqXvdj/SX6ZE/Z2LVFtqJZaxd4use+EvStZ+l4rse6E2yr57fbWthPSK1nf2ytMoWUnDPX5a9pt6c9JaJJfwQnDK+f64/PKNemW5ldzwrDC9Ze97u3jVXbCsGJ+HsCKE0bisN9/DOtbAbDjhABADcbJXpQHsOSEAqDFDfNJHsCWE0biunKDcbKXA7DmhHBMbjBO9s4A9p7JYQqwGciacvq0WokPD4S8yc6MsOfVCnqbHIBFJ8gAhp6sJX0KAvmJbNKOd+DLQPZGZwCbTqIABACQ5pJpBrCgy5RpeAKw6kR5gAAAPBUgAABPAbDrZDkAcaaXfKYCBADgKQCWnfAMEAAAUwFEa8HnKoBlJzwBQH4wJwoAtBZMBYgsO2EGMNhDxmREDv0J1Jz4/Qm09hsykp3JTAK0I8tOmAIYXtdIPBMjy04oAUznFTy5IstOCABUrH/7fV/UC5nO4Wf/H6PLPfTmG/IiO/03wmIAQHDCkNLX9F4/T0L+nK5/3gM5TkLGAaB7+3jkJ04IAJ4KENDnbE0+A3AaIzlhSPlQBQgAwFMBXqkAwHHCkDMVIAAATwUIAADJCcXfGgoArD98qgIEAoBHeE6YAfwFALkmsiPAQQKIziOqE05Hsg6EvTxCiWVvIDsDQg6jEbS+frcT+jMpYBvGn6R/HQh5kJ03cXZAyVazDaoTjrI5wBfp+jM5KxkDJQsUJavFCU93wdEJh40TNk6I6oSDdP0bksMkW5P9vszffx2fEzNUJzQ7r40TIjqhUT6mExrlYzqhUT6mExrlYzqhUT6mExrlYzqhUX7zf8Jf/X9Co3xMJzTKx3RCo3xMJzTKR3PCcSxrDBWrpfYSJCcs/54mVxhOWCUfxQk1Wev397WeCcMJi/k78Xjo7rTnBMMJi+c6gXEf2muC4YTFcx3DuJ52Tth2Qqqdax8w7lM7J/nt45GqTki22u+uE0qTli5/W55f8X3CcUs719Zr/T05Nsiv+D5hsjb/7njXM8mv/D7huOTdlVMvZuXHa94ndP0+n+v8Hz6Tm/wm/3fnl79PiJ3f7DFxkd/sMWn2mDR7TNLPBSesZ9/dVSd0vu/Q9b5Lx/mu990633fsNt/1vnNH+f8BpFKD0GeaAkUAAAAASUVORK5CYII=",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.insights',
                        style: {
                            'background-image': "https://cdn0.iconfinder.com/data/icons/light-bulb-filled/64/lightbulb-03-128.png",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.likes',
                        style: {
                            'background-image': "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTM2PzmgU_jqG0O9btYn9Hovoy68I0r4K35VKPHvji1miVfDkqR",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.dislikes',
                        style: {
                            'background-image': "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSgnEJxx3fGXHTjp0daFImWnQLJqJ6wR3HXV-OWOW5Gr2Ft2iQw",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.views',
                        style: {
                            'background-image': "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQSZd5JxL1YSSCygfKqFOuQDRrC-aHwBwQgOZKIS2L5_O-_UQDg",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.barclaysApp',
                        style: {
                            'background-image': "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAMAAAD04JH5AAAAn1BMVEUard/////z+v4isODr9/32+/7p9v36/f8Kp90Vq970+/7v+P77/v4Oqd3g8/za8fzT7vuT2fDk9P1cxehJvubc8fw1t+PK6/vU8Pl0zezU7/zE6fo8ueSw4fqF0+5/0e245fWq4PNmyOqO2fi35fp4zu+d3PGi3fcAodtLwu5+0vYkw/N0z/W/6Pul3/mv4vRm0vU8yfNx1fYbvvFbyvTJba4dAAAL/ElEQVR4nO2b65aivBKGCeEUMIBCAwp4ANSRAXrmG+//2nYlARTtblH78GdnzRptJLxPqiqVBIKEzsoE/kW7zA/wl5TAz3YR05icaUoDeW3qY+lri7+0udQ1wATZ869WFyVzzgikXn4y/R55KHip9whS33z/u+RZSaKOQGr1w29rvih43RJIQn/9zfpQNoJA4vrx9+tLkrABt4D9E/oSjnoLfGv8nYpvcoAJmv6MviStQFx63AGEUErJUwTgBGmC5o+Igzb2Mxg2CH0CIGMueMAAlAZe5m1tVUdqNH/GChEA3B8B1F9rqq6HmqVYso5eH9eHKJDQ3V2AzlWkq5alyhYURUPLx72QqFJ0rwdI4NZFvbB0S+YEluXix50QS7t7q9CsqaqqzGNVllsb+I+bYCnd3QdIcoDJzaGqCl0QKFZmPAzgSWNDgHW87ivLAbgqHUGgWNtzC7D0MN4liRSMVMeJn2ByujIJykKEgaIszyxAYOrnB5IxkgFLY2KQ4vnaceI0r4LTdXHVsCgAAu3UDQg+5oVra9E0GBsXIwAM31VVWbPdel8eTodPANPeOUHVhAtnsbBlZTWS4DaAkSmaoups8Db3VdLagOCy4ACype37Y1UzA5doM8d1lek4L9wEoP7LwkI6FFVFWnnsK+a1zgFkrWgPkaR0LU3TZrZtO/FinA1uAuAiFvpQkFVWvdg+7ADCNjTIoXQ4wMx+cZy69sfY4BYA2RZ2p6+jojx25q4aTTc5gPKS0RaqTBVhAgBYFPmYFHkDgAT7mOlPoCC5znNxPmbHdVMAyHbRKR3LJhQAQBDvD58AkDXWROhPkDoLMe/yxMB5oQCAINDcqcSlCN3uY6v1wWLRlCNMcAMAlzVq2896gfLqgwrBXlPbTL8zQbhPIAVKONsXiohCBlDnybMAJMkhAib9ela2ZvVmv6nDEAJA7QAseRYW+Wqb79NaOwNw8+PTAMe9ygzQryV1yDtgYJiQqOoJwJKtFzeO49BhWaBzgbvYV7d9cMMFx+LcAHwRy0KSJQVO0AIwBP6fMgAoznP3QwC4Ci8AJl2XHFhAFIWXE4AbV7eD4EMAyO0v9wBYA4CFG5a3O+LHAEmpoUEMDADUWwDu+eD1EMChlEUW6PUnA/33AGzhAjd/EkA6lKZ+SgNn+m8DKNcAN7vBDYCcq036oo8DaINwkR8/B6BHGOh3ISAP9M8AQshEt0fbcQAcQdfHGUAMRhzg8HkAg6J+GIPtcPzJFrgCMC88cNELOws8FQPkkJvjDXDRCT7DBZCIZF39WP+tEGg7Qfh0HmCZ8ApA7fSHBrgOATcMn86EQWlPVPVa/qT/lgd6gLh6NhPiyoWJh9ozqO/qnxugG4rC+unREFcw+Vevylv6V1kAAJ6fD0jHVFjgUr7VvzTA0ANhczsN3JqSHfaKal7Ky2/og3ZvgLYPhOH++OHVRwDQJHf47Pes7ZAZ5Df0LfnKAHE+YmFwe1qum4OiKvuFfqEPAJaWh7J2loVYCJS3Q+D2wiRV2oa3sac7x6MgGOpXvwuxLpzZrQHCtBqx+L8FkDRte/WwlS1ymCedyzMHqM2hStnKdKbFzYswwKcszSQpr4W99T8bi3246fZQzcyBvqXI+fHYwKpoNgvzMna4AT5lcSpRr3BUaK8e/1tqimxaaYMP5WxgfygAgLfpQpnZTVk2wgBF9RnLcwmnzASmUv3zbUVWwzSjx9y60NfMtAqCtJjN3NT3i3oBBqiLESvDMXdItjWYQG2Okj9T1Fm6l/CxUS8zoAzjjrFNY21RZHRfxzAOxPmoezQjUlUdW2qYH4y5rMhF6huHMlQv9BXFzisJ79PFS53TJC7AA+GoGyRjblJt3YVdbwndyGaYbolU5cppHtpNA6y0DKifpnaYYurVYIPmk25SQand2RrTwJHtdI/ZHUr1lP+6Mchy84yQbVovYg8+i/plnAHGAFBfnQWEznUtTX0KBniRlYH5+RCgNdDrwAmxy+7abWfTT7tPCARL0CWhWqQZkYK8MJVB88UQpMV7aHqQNrGbgPZq7D38UY8LKMFGhmoWAGTb2DKXtQbthwwM/pGInzbujrIq4/THAbCpiQ0pED79Jjav5Zm+HTZbilkgOvc8QBgJYCwt1n7oaYVmKZfuZyMQTIOgjxIg2Ifr8fpjAehqzzwL4b2QlQt5Tcg7jsucAMba7u94iDP2kRGRmGM9SEqX8rz9TH+xiIucP80g4/VHA3CIoI61t+Rbfdct4uze50d3ABD8Gs4UbSDPzc/nYGIOUIf3PsG6ByDJq0a2tF68k2+bD0uxosy9O5/h3eUCfDyWtnwtL+aAbghzgTGToIcBOMGxkJVZ53uQ783v1nk5Zhb6DACUw/Hvf7Y1O5MX5nfDtCxH3Jl9GoAEh99/G02zXwbyvPl3m/8RAEgIyd/ff2pt5jgn+Xif5480/xEAFgmH37//wPzPEepu3OR5mdyTfZ4DYLV+A0Ljvrx08r706BP0xwAIR/j7XxMXzR7kn3iA/+gWIkD4Bwx5mm+Th1v/DIDEn1P/22bBU/JPAYhx7zn5JwE+o/wf4ARADV7Oh3Pab4Qg7Y9nWyioQSnlp0PNYSTA2fzT6H+g4oioMzi7ByD+kpW5fxZW/rSdXpCA/7jKgv6KOJtuXndLn03F5xfbYYP5ktfK4Ad+Pk3m84AQ6i83r5uld352/92Ytpsso25INTyENrT/Kh7atNt+6FxpnyNOKdREg3kQiZDOprDGEqF2f0mEHFg3rds6u7OzBwCb5dRFKO4auUYTRbSAAcTL5cZEiC+5mKa5mc+Xa92j9AKA+khFU4MlKwfxCxgrOMMgr0hfQ51Xy3sHACW/KNCbQWs2HelobnQA81/U8CfI5u0HQyU8KAJRcwCwgXo2/5YBsAHrWQVMSeB6UwMigA78NQCAdQV8tADsm4VC2gGsDAnTiDUJlklwkrgOuQIggYWUCfIYOQnhRInukAX/e+w0LOq8BwBBGrYuYCo7cAu/tADAFCvI6Rs2qHkCAHubmYJejbbe7pevQyjAkgmh9fX2w+FlsL9GirgYU/ETxJ3ZuQC/wvVg/bdD6MyNly6IUGysWztCHOnJGkW8RRo4LiMX+74GQWhDkCm+OIOyahARNo8iAHhdvULkO1ABrKSezT6HAKyh818Zd5n4C67K/cGaBAhzaWCFYTec6BDdfAMQBbutKAtfttbpuqG1w5R3M/N9gA0wU/Bf1P2po3Wbl3yb7yIe9NmBCzKceKHoaSwEE4wTE61bgHBqIpv78CMAEphoRwKya4+xv9Wk/ZVKc76Z+P1uaBADPMUDXUM6W4LpLKdgEQPTNjFBbKPknRiA5DPhd08g3YgNLxGKTrCUeDYSbn0LAHoJBt9DdzTm3fb3CYtg3gso24bL4MGsKHsvCKP+Ub+C2wPOmcUlA0cDE7wFAF2PxJBpWPG5M0U3pCwPYcgGEE1xH8y4rSmuwk4M/MQPli0kuQBgN3vO8S97s+GrLIhYKBtsayI7DNlW5AFDOIGlCMYDvxsYdzmUJUZCXyEHsg3PFFsoHgLAGMkGul+rQa8dAHg4mWvM2MauNSBmCXTTAXADe9wj4GI/SLypyQejyY6PltMAQm7JjYGNjRiRWAwI/WDqBWA/T3k3BiYq226OoBtC8u5GLBpDyLepmHdrJSCYDQYIqSxMNNyPo9CyVR/yjJmPSJ0FKLRc12zlvV5Al6pimkq0A/PQTOs7K3zXMsOztTmfeyw1hfVS6rOshMxo6sOVFceJWLH9V/k01MbKmjV8zT8Y0dpmoMr67TwAX1khgynRdSFtFbHLNPjo9Mv5MgRMAGGNh1XwsMKYSXa3oe/+zfVvbcj/8UnpqG29X1fw6I3NX1QCKftZAO/+ze2fW5ZS9LMAa+mn3vERJTEltPxJAPaKh/OTmSBir/n8YD/w+Kted79l8nklRBP2qtePRcFcvOqF1B/qCInVve73Q04I2TxGvPC4/gn9zemFxwnafL/+7vTK54+8dLo5f+mVEUTJd8oH4fC1X0Zgrr5Pf25evvgsjOB9j7wXnb19Pnz5PVp9uSOCefTOy+/itXw1XLLly5cUHHjL2Lx4/f9/SCkU3sO75cwAAAAASUVORK5CYII=",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.macysApp',
                        style: {
                            'background-image': "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxITDw4NEw8WDxAVFhYWFw8WFg8QFRoVFREXFxYYGBgYHSggGBolHxYVIjEhJSkrLi4uGR8zODMtNygtLisBCgoKDg0OGxAQGi0lHyUtKy0tLSsrLTUrKy0tLS0rNzcrKy0sLS0tLS0tLS0tLS0tNy0xLS0rKy0tNSstLS0tLf/AABEIAIAAgAMBIgACEQEDEQH/xAAcAAEAAgIDAQAAAAAAAAAAAAAAAQUGBwIDBAj/xAA7EAABAwIDBAYIBAYDAAAAAAABAAIDBBEFEiEGEzFBIjJRYXGhBxRCUmOBkrEXYnKRIzOywdHwQ2TC/8QAGgEBAAMBAQEAAAAAAAAAAAAAAAIDBQQGAf/EACgRAAICAQMEAQMFAAAAAAAAAAABAgMRBAUxEhMhQVFCodEUInGRwf/aAAwDAQACEQMRAD8A3iiIgCIiAIiIDiVxc4Dj81yKxzbzEdxQTkHpOGRvi7T7L43hZJ1VuyagvbL6KVrmte03a4AgjgQdQV2rE/RriG9oI2HrRHdnwHV8lliReVkXVOqbg/TOSIi+kAiIgCIiAIigoCUUJdASii6ICCtW+l3Eby09KODQXu8To37FbPkdYXK0BtHiPrFVPPyc4hv6G6D7X+aovliODZ2PT9zUdb4iZN6KMQyVMkBOkrLgfmZ/kHyW2wvnfDa4wzxTj/jeHfLmP2JX0JBIHMa8ahwBHgdUolmOCe/afovVi4l/h3ooRXmGSihEBKKApQBQpRAeLEazdRmXI6QDi1gzOt3DmsWHpKo/ifR+6zNwWH7WbERVIMsdoZ/eA6Lu5w/uoz6vpOrSdhy6bk8fKOH4lUXxPpT8SqP4n0rVWJYdLA/czMLHd/AjtB5heVcrumj00Nl0k49UW2v5NoY16Q6d9PKyLPvHNLW3bYC+l1q8BEVc5uXJpaPQ1aVNV+/kELZOzO38ENLFDNn3jBl0bcEDhqtbKUhNx4Gr0Veqio2ejb34lUfxPpT8S6L4n0LUC9FFRySvEMbTJIeQ7O09gVnem+DOnsmkgstv+za59JdH8T6CsnwrEBNEJgx7AeGcZSR225LFtk9hI4Ms01pZtLN9hh7u096zQNXTDq+o81rP0yl00Zx8s5hSoClTOMIiICLKLLkiAq8ZwaGpjMUrMw5H2mntB5LUO0+x81IS8Xlg5SAat7M4H3W8CqLbLEBBQzyc8uVv6naD7quyuMllmlt2tuosUYeU3waJRQ0KVwHvEERCgL7ZrZSerN2/w4QdZnD+gHiVt/AMBhpY93Ey3vPPWce0lVvo9r97QRe8y7HeLeHlZZQF3V1xS8Hhdy1t11soT8JPggBclKK0ywiIgCIiAIiIDitZelzEelT0be+V/wDSwebj8gtmFaF2uxHfVtRNfQOyt/S3T/KpvliODX2SjuajqfESnAVhguETVUu7hZf3nnqtHa4/2V5spsTLU2mlvFB4dN4/KOQ71trDcMigYI4owxo5Dn3k8yqa6W/LNrcN5hTmFXmX2RozHcDmpH7qZmh6so6rvDs8CqxfRNfRRzMMckYkYeLSP91WqNq9hJIM00F5YeJZ7bB/6HellDXmJ82/eoW/st8S+fT/AAer0S4jlnnpjwkaHt/UzQ/uCP2W1gV894DiO4qYKkcGPF/0nR3kV9BRuBAcOauolmODK32jo1HcXEl9ztREVxiBERAEREAREQFTtHO9lLMY2l8pblY1vEudoFieyPo/ZDaaotLL7MfsM8feKz8tSyi4pvLL69TZXBxg8Z5DWrkllKkUEKCFyRAYHtdsHHPeaC0M3Ej2HeI5O71fbHySGkjZKwslj/huae1ugPeCLFXlkDVFRSeTonqbLK1CbylwSpXmgqWPc9jXtcWGzgCCQeNj2cV6VI5wiIgCIiAoNpsadTeq5It8+aXdCMHKS4sc5tjy1GpPAXR8GIhucT07n8dyYpAw/l3me48cvyXLHmj1jDO6dx+e4kV4gMWw7Gp6y7YGimEfRmfK0yFsw68TGgjNl5uJtqLL1CrqIZoo5yyaKV2RszGmNzJLEta9lzcOsbOBFjYW1Xbs03SstzqZvuF0bbNd6p0f5m+gyW45t80N8ygIZX1FQ5/q2SKnY4t9ZeHSl7mmzhHGCOiCLZiddbBTUz1dO0ySZKuAavyMdDMxvNzWgkSWFzYWOml17NlHNNDSZOpumW+kX87q0JQGL0m0j5o4mwMZPNLvHtOYiJsLZSxsr3AXsQBYDU6r2+rV4FxUwSH3DC+Np7sweSPGxVB6MzHar3VshLDFbh6vmkEWX8vW0WdIDG8K2jMlWaSSPdOyO6BN3NljI3jCeDgWvje13MZuxXldVMiikmkOWONrnuceTWgknyVHtOxuV9TGAamldHO5rbF5Y0Ou0+LDIApxuRtQ6ko29OOa08luBgjyuA8HOLB3i6AqNmKeaR9VVRQw0Esjmme7DM98pbnDT0hlDQ8AnmSdAsowWqke2RszMksbyxzgDkfYAh7L8iCPA3C80+Eu3sk9PUbh77Z2lrZonOAy3Lbgg2FjYjgu3A6+STfxTRtjnhcGOyEljgRmY9t9QCDwOoN+KAuEREAREQFdiFCZJKWTPbcyF5FuteNzLd3W8lYKUQFfhVEYhLd188r5OFrZjwTGKEzMYwPyWlikva/8uVr7fPKrBEBj78Mnie99LIzI8uc6klDsmcm5dG9usdze4s4Em+mt+M9BVVHQndHT0560MJkkfIDxa6Vwblb2gNue0LIbKUBjEezWRkO5lEFRFvBHK1l2GOSUv3T479Jg00uCLXBC9DziLhkApY/+xmnk+YhLR5vV9ZSgKvCcLbC0i5ke85pZn2zSOtYudbuFgOAGio9gMODIpKkPzxyOLKfS2Wkjc7ctGuoN3OvzBbfgsjxSk30M0Gd0YkaWl7esA4WNu+11300LWMZE0AMaA0NHAACwHkgKNmH1EMkvq+6lhke5+6ldJEWPfq8te1rszSdcpA1vr2ezBMPfEJXyyCSaV+eR7Rkbewa1rASbNAFtTdWyIAiIgP/Z",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.riskPreference',
                        style: {
                            'background-image': "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRysGRsXxSl6bcT79jVkr5T8DV4Ztir38wH67UgvfIFThVWgr-lyw",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    },
                    {
                        selector: '.insightTypeAffinity',
                        style: {
                            'background-image': "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAMAAAD04JH5AAAAq1BMVEX///8yRlT09PT+/v719fUzRlO4y839/f254+3/urD5+fmxw8X6+vq1vME1RVI2TFey2eP0s6mutbnU2NrJztJAVGC/xspRYWxXZ3NndH6Mrrqdw867k5Gly9ZDUFyAoa3Sopzi5ORufIbiqaFRV2FUbXiDj5iRnKPT3+C1j42bpKpEV2Oou76StcHe4eSPmqKNoabB09N3iZCdsLVme4OQe36jqq9/i5FohZFOeWWBAAAIhklEQVR4nM2bbXuiOBSGJYC2CqUFFOx0ZwZ3Sx11tLYdu///l21eIZCTENC2m0+9DiH305CcPAkyGv3fikvKF8REmZDyBTEhbRIEwcT99Fh1LSDF/fRYfW06nSr14VhyOCRD723HqmtTUlyrWHzjOE447N52bDDfuQmH3Kvne56n1IdjjF8r6HNvO9a45rXqe3BM8IUCXT2bmMxHSn2EoFjNZwp09WxiI1fwp4Debj5RcA7f5fmAzI2+/EgoiM/hMwF0brbHSxf/2zf2h48VDOWLjDgJBvCvrqgC38cKBvOZAJybh/CpAp+Um9DmXiWGM2JA1iS6OAzhYwWUL2ekXvwp5VMBw/hXt78pv5ETe/D5miCG4gD+LVbQyonWc9Kr1iS3J38u8W9vv7VyojXfq8ZdOzX14vO50D8n6teEnnxZQZ+ceDm+pOCMnHgOv1KA88FH8/0fEL/OidHiY/m43EP8OidG6QfzmQKFf/WXw3OirODiz3/DFRj4soKL85NRyBRo+LyLhILe/KnXxR9RBb5/D/If7xsKBvBBT9jgEwWEdQ/xb69+CwXJ5Txhiz+axLSz7yH+VaVgeTFPqPA9FNOHfQ/xr67+ZdW3l/KEAB8HpXzQ4v/gPbC6kCcE+d4klBSA/HlyGU+o4Y9GtYLz+B2eUMuvFZzLN3pCA18o+HEu3+QJQ9/AFwrO4Xd4wtT0/wsF5/GNnjD53sEfjWLKz4byOzxh3slHuV/nxAF8syd8t+bT2TiEb1yTQ95g2sFn/XR/cX6xZQ3GHfylJ5Qa+PvNJunrSfgDeHe7+GI2GviLCD+nzr1z81rCGsy8br6kwMDv3Du3ri3lAdDBH8UW/K69c+taylosL8g3752r+twTsg6Yo0vyTXtniU89IR8Bpz787518/d5Z8YQl+5e8y/Grp2DlCT12+re5HP/hp1AA7Z0VT8iXWXQ5/vU1VwCdJ6qecEXrvnTzT9Z8rgA6T1Q94ZS1uoD43lA+VQCdJwKekCWBeQDw3dVgPlEAnCdCnvCJ3rcC+KPTGfzru5/qeSLoCVkWigF+EZH1dyj/7u6nkhMhT+iySonKn7xj/rwYzG8r0HjCglaJAP6ePID4DH41G0VOBD3hgs7XXOV7G2JAadB+/jf5sgKtJ4zpfH1X+SQFsPVhYeY7er6sQLcmPdF88aTyC9IuGQH7yIr/B+LXOTFaaNbEkuaLk8JHC3xnTv7Orfh1DzT4dU5snyeKQnON30hXLCeSgfc+qgaAZv13pPX3AeJf330HzxOrNqgAnIjbfI8kKLxCTplfjlIj/1Ap0PKb54l1G0t6TRbAcyJp81RZ0JORv+DVHkx8SAGeGyt6TRIgcnLJBLBEmXfwhdBfIP+fX/w5tRUQT7jynYaAak1gjwCxMRya5j+7me2df0H8v+8eHEgB9YRsSxIrfJr9Sp6ntt18cZ74C+JfXwsFOb1Z8IknZMNno/ApOh9tRJ7q5IvzxAeIf339hyckxROyNF8qfNb56IWOoZMNX5wnPkD8ugfannBR90zLE2U4Hi5pvyrvJATfl/niPPFBz48SxRPu2QVX4dNRuFzSOZTa8cV54oOWn6qecMIuFaonpdKoKXRSS/6ozgcaPuAJWaoPAU+K7bL/neaQ2JZf5QMtX/WEbBpAnjCpclgp8VF8Y+DXu3cNX/WEbBTOkcaTUda2GoRusvKN/Po8UeXD54Qeq78APFGRibVuFRb4BrcIl34XX+REiA+fEy5p/RXgiVAyd6oSZduoXlcM/GDl1DlR5uvOCWNWv1D5yJMVONK6Nk+b9QA+zYkNvu6cEN3Q+ieAPxkVOcTP9x38LcuJTb72nIJ5knkA8FmDLX60CdR6DX5WxJLuLr6Xsnalyd5oN8kb/PkBwfVkvsiJVnyEDTh9rp6uXW+znDN+flhIHWXgS6d53XxxTnvQtUtiqEj24DjV8KvdhAXfE857r+fbxAbzXX5G4GTBZ/Ord8d8qJefzhfvjgu+/Qo/hd8+JyRFzNv04/nwu2OXPQQ/Sj6eD787RluWa+b7D+Zr3x0nfK3jhzJ9+NNefO2744XItZICKz5a2vON745PYq3Vr3VQjCyY9nzju+ON2Ovr392psXRO+Lkt3/x7wo1YQGJb/vTg2PMtfk9YKVghK36aOZKf6+R3/57QfRIK5ifIJzf5xQvNHf7Gjm/1e8JagZ/FZn5R8nopi4VdfLvfE7oHzsdlu0FyPZk/WfD1yy8LGnNPnXzL33MxBcL/rcJKQ82fpiV/1evkKYuJd6/n83GglPwvYZRhUr3PQUn4tBRPyclirinJBH/hnc1HxYvM52Myy3HJGtuEPOTjNKgSSPQ6ns3GyRq5Rob+GirW4/F49tLmN/uEKioT/kzcRVYFCR8X3Ma6QL35aH0ck4JbOEQVGOJnJTbHnJ9KT2RX8WlZox78SXHkt9E2XrdO3e4yiyp+tnqiG1XGL2Jp5/Q2a/JxORYTy98TFtU9vI3Zm9TfWzwOSSnqZ0vO0+NVVNeJnscKn3WDhSecrNt8XJ6rEUePmpebxX4q7kRJXOaNMfG4g/mz2dHCE0J8MhaV548nwzLP5X0676LnsY6PFXR6wgDk47J7A8a/Oibn5czAn42DJl/1hBMNn0qIuvhbijfwGwJgT7jW8XFs954Z+NHbs+FeFls3+ZAndNfGNnblI8ifvz3PmiyQL6dZrSdER1MbuLyWb/m8nhPbx5fnHVSvHTvKC6rJE+JEZOLzstu9Pj+/vu666onYsXBbfKMnXB8t27WL8VQsMUyekH0TFxSSiHP4OAn38oTyN4EBWlMVQ/nHNc6+fb87VH5jPMELc/uJdPGPGF2gCdje4O8+A7ziYSnrIy4M1uxoUjC2QCiwag/k9/l21pvywvLJRb67/ervjr/4u+uv/u78i767/w9hdwqRIv6+OwAAAABJRU5ErkJggg==",
                            'background-width-relative-to': 'inner',
                            'background-height-relative-to': 'inner',
                            'background-width': '128px',
                            'background-height': '128px'
                        }
                    }
                ]);
                var elementToDraw = document.getElementById(elementId);
                elementToDraw.style.width = "1200px";
                elementToDraw.style.height = "1600px";
                var cy = cytoscape({
                  container: elementToDraw,
                  style: fullStyle,
                  boxSelectionEnabled: false,
                  autounselectify: true,
//                   layout: {
//                     name: 'concentric',
//                     concentric: function( node ){
//                       // there is node.data ...
// //                       return node.degree();
// //                       console.log(node);
// //                       console.log(node.degree());
// //                       console.log(node._private.data.level);
// //                       return node.degree();
//                       return node._private.data.level;
//                     },
//                     levelWidth: function( nodes ){
//                       return 2;
//                     },
//                     equidistant: true,
//                     fit: true, // whether to fit the viewport to the graph
//                     spacingFactor: 3,
//                     zoomingEnabled: true,
//                     zoom: 0.25
//                   },
                  layout: {
                      name: 'breadthfirst',
                      fit: true,      // whether to fit the viewport to the graph
                      directed: true, // whether the tree is directed downwards (or edges can point in any direction if false)
                      padding: 30,    // padding on fit
                      circle: false,  // put depths in concentric circles if true, put depths top down if false
                      spacingFactor: 1.5
                  },
                  elements: elements,
                });
            });
        },
        render: function() {
            this._render();
            this.model.on('change:elements', this._render, this);
            this.model.on('change:domid', this._render, this);
        }
    });

    return {
        CytoscapeView : CytoscapeView
    };
});

<IPython.core.display.Javascript object>

In [12]:
def update_and_return(d, ud):
    nd = {}
    nd.update(d)
    nd.update(ud)
    return nd

def pop_data_in_nodes(graph_repr):
    new_graph_repr = {}
    new_graph_repr.update(graph_repr)
    new_graph_repr["nodes"] =  [update_and_return(x, x["data"].pop('popme') if 'popme' in x["data"] else {}) for x in new_graph_repr["nodes"]]
    return new_graph_repr

def pop_data_in_edges(graph_repr):
    new_graph_repr = {}
    new_graph_repr.update(graph_repr)
    new_graph_repr["edges"] =  [update_and_return(x, x["data"].pop('popme') if 'popme' in x["data"] else {}) for x in new_graph_repr["edges"]]
    return new_graph_repr

In [13]:
import networkx as nx
from uuid import uuid4


# Set the id of the current element
ID = str(uuid4())
set_id_for_dom_element_of_output_for_current_cell(ID)

# Make a graph with networkx
G = nx.Graph()
G.add_node(1,  label="bob", level=10,                               popme={"classes": "bottom-center profile "})
G.add_node(9,  label= "Insights App", level=10,             popme={"classes": "bottom-center "})
G.add_node(2,  label="assets", level=6,                             popme={"classes": "bottom-center assets "})
G.add_node(3,  label="General Preferences", level=6,                popme={"classes": "bottom-center multiline-auto preferences "})
G.add_node(4,  label="Application-Specific Preferences", level=6,   popme={"classes": "bottom-center multiline-auto usecases "})
G.add_node(5,  label="friends", level=6,                            popme={"classes": "bottom-center insights "})
G.add_node(6,  label="insights", level=6,                           popme={"classes": "bottom-center insights "})
G.add_node(7,  label="likes", level=4,                              popme={"classes": "bottom-center likes "})
G.add_node(8,  label="dislikes", level=4,                           popme={"classes": "bottom-center dislikes "})
G.add_node(10,  label="views", level=4,                             popme={"classes": "bottom-center views "})
G.add_node(11, label="risk tolerance", level=4,                     popme={"classes": "bottom-center riskPreference "})
G.add_node(12, label="short sell insights affinity", level=4,       popme={"classes": "bottom-center insightTypeAffinity "})

# for x in range(3,100):
#     G.add_node(x, label="{}".format(x), popme={"classes": "bottom-center"})

# What are the different kinds 

G.add_edge(1, 2, label="has",                                        popme={"classes": "autorotate"})
G.add_edge(1, 3, label="has",                                        popme={"classes": "autorotate"})
G.add_edge(1, 4, label="has",                                        popme={"classes": "autorotate"})
G.add_edge(1, 5, label="has",                                        popme={"classes": "autorotate"})
G.add_edge(1, 6, label="interacts with",                             popme={"classes": "autorotate"})
G.add_edge(6, 7, label="total",                                      popme={"classes": "autorotate"})
G.add_edge(6, 8, label="total",                                      popme={"classes": "autorotate"})
G.add_edge(6, 10, label="total",                                     popme={"classes": "autorotate"})
G.add_edge(3, 11, label="is a",                                      popme={"classes": "autorotate"})
G.add_edge(4, 12, label="is a",                                      popme={"classes": "autorotate"})
G.add_edge(1, 9, label="uses",                                      popme={"classes": "autorotate"})
G.add_edge(9, 11, label="used by",                                 popme={"classes": "autorotate"})
G.add_edge(9, 12, label="used by",                                 popme={"classes": "autorotate"})

x = CytoscapeWidget()
x.elements = json.dumps(
    pop_data_in_nodes(
        pop_data_in_edges(
            nx.readwrite.cytoscape_data(G)["elements"]
        )
    )
)
x.domid = ID

# Use this as the starting spot of figuring out what to plot in the profile ...

# What are the different concepts found in the profile?
# What belongs in a profile?
# What does a profile look like?
# How do we change the profiles view with time?
# How are my profile attributes being used?

# Maybe there are roll ups to the profile and leaf nodes ...
#     How many app specific attributes does a user have?


display(x)

<IPython.core.display.Javascript object>

CytoscapeWidget(domid='2030aa39-7c63-41f9-8886-28e59c8661b9', elements='{"nodes": [{"data": {"label": "bob", "…

In [8]:
What happens if an app requests specific information about a user but its not there?
    WHAT SHOULD WE DO?
    - Note that specific information was requested
    - Only let cons

IndentationError: unexpected indent (<ipython-input-8-c0891181c1f4>, line 2)

In [9]:
# Users have general preferences an application specific preferences ...
# Some applications may use general preferences ...


# What are the different kinds of preferences?
#     Do preferences opperate at the global level?
#     Do we have preferences per application?

In [10]:
# Should I add a profile slider to show how a profile has changed over time?

In [ ]:
# Proprtize features to build a wealth app
# Should this be made into a skill or directly added to the platform?

In [113]:
# Gantt chart demo!
import pycandela

data = [
    dict(name='Do this', level=1, start=0, end=5),
    dict(name='This part 1', level=2, start=0, end=3),
    dict(name='This part 2', level=2, start=3, end=5),
    dict(name='Then that', level=1, start=5, end=15),
    dict(name='That part 1', level=2, start=5, end=10),
    dict(name='That part 2', level=2, start=10, end=15)
];
pycandela.components.GanttChart(
    data=data, label='name',
    start='start', end='end', level='level',
    width=700, height=200
)